In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, ZeroPadding2D, Add, ReLU, LSTM, Bidirectional, Input
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
data_frame = pd.read_excel('C:/Users/admin/Documents/DPL302m/DPL302m_Material/Code/EduNext_Slot5/data.xlsx')

In [3]:
data_frame.head(11)
df_preprocess = data_frame.dropna(subset='Unnamed: 0')

In [4]:
df_preprocess = df_preprocess.reset_index(drop=True)
df_preprocess.columns = df_preprocess.iloc[0]
df_preprocess = df_preprocess.drop(0).reset_index(drop=True)
df_preprocess

,STT,Thời điểm đo,NaN,NaN,Điện áp (V),NaN,NaN,Dòng điện (A),NaN,NaN,...,NaN,NaN,Tần số (Hz),NaN,NaN,NaN,I (đm),U (đm),P (đm),Mã trạm
0,1,01/01/22 00:00,01/01/2022 00:01:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,237.234,236.063,237.32,304.2,348.75,334.35,...,NaN,NaN,50.13,50.12,50.12,NaN,811.59,230,560,020343
1,2,01/01/2022 00:30:00,-,Serial: - - -\n- TU: - - TI: - - HSN: -,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2,3,01/01/22 01:00,01/01/2022 00:57:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,234.406,233.089,234.427,279.6,297.9,286.05,...,NaN,NaN,50.1,50.1,50.1,NaN,811.59,230,560,020343
3,4,01/01/22 01:30,01/01/2022 01:25:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,235.978,234.657,235.904,260.25,270.3,277.05,...,NaN,NaN,50.02,50.02,50.02,NaN,811.59,230,560,020343
4,5,01/01/22 02:00,01/01/2022 01:54:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,233.935,232.942,234.02,294,279.3,287.85,...,NaN,NaN,49.99,49.99,49.99,NaN,811.59,230,560,020343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26663,26664,20/06/23 13:00,20/06/2023 13:00:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,232.302,232.09,233.033,413.7,455.25,441.45,...,NaN,NaN,49.83,49.83,49.83,NaN,811.59,230,560,020343
26664,26665,20/06/23 13:30,20/06/2023 13:27:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,233.709,233.969,234.602,417,439.5,460.5,...,NaN,NaN,50.03,50.03,50.03,NaN,811.59,230,560,020343
26665,26666,20/06/23 14:00,20/06/2023 13:54:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,234.291,234.486,235.393,411.75,434.1,393.45,...,NaN,NaN,50.08,50.08,50.07,NaN,811.59,230,560,020343
26666,26667,20/06/23 14:30,20/06/2023 14:22:00,Serial: ML31717088282 - ML3 - Meter NURI\n- TU...,235.209,235.167,236.066,407.85,449.55,438.6,...,NaN,NaN,50.04,50.03,50.02,NaN,811.59,230,560,020343


In [5]:
df = df_preprocess.iloc[:, [1, 4]]
df

,Thời điểm đo,Điện áp (V)
0,01/01/22 00:00,237.234
1,01/01/2022 00:30:00,NaN
2,01/01/22 01:00,234.406
3,01/01/22 01:30,235.978
4,01/01/22 02:00,233.935
...,...,...
26663,20/06/23 13:00,232.302
26664,20/06/23 13:30,233.709
26665,20/06/23 14:00,234.291
26666,20/06/23 14:30,235.209


In [6]:
df = df.dropna(subset='Điện áp (V)')
df = df.reset_index(drop=True)
df.shape

(25277, 2)

In [7]:
df['Điện áp (V)'] = df['Điện áp (V)'].astype(float)

In [8]:
df['Thời điểm đo'] = pd.to_datetime(df['Thời điểm đo'], errors='coerce')
df['Year'] = df['Thời điểm đo'].dt.year
df['Month'] = df['Thời điểm đo'].dt.month
df['Day'] = df['Thời điểm đo'].dt.day
df['Hour'] = df['Thời điểm đo'].dt.hour
df['Minute'] = df['Thời điểm đo'].dt.minute

In [9]:
# Check for null values in the 'Year' column
null_years = df['Điện áp (V)'].isnull().sum()
print(f'Number of null values in the Year column: {null_years}')
df = df.dropna(subset='Điện áp (V)')
df = df.reset_index(drop=True)

Number of null values in the Year column: 0


In [10]:
df['DayOfWeek'] = df['Thời điểm đo'].dt.dayofweek 
df['DayOfYear'] = df['Thời điểm đo'].dt.dayofyear
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)


In [11]:
from sklearn.impute import KNNImputer

# Select the columns to impute
columns_to_impute = ['Year', 'Month', 'Day', 'Hour', 'Minute',
       'DayOfWeek', 'DayOfYear', 'IsWeekend']

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer to the selected columns
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

# Verify the imputation
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25277 entries, 0 to 25276
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Thời điểm đo  25277 non-null  datetime64[ns]
 1   Điện áp (V)   25277 non-null  float64       
 2   Year          25277 non-null  float64       
 3   Month         25277 non-null  float64       
 4   Day           25277 non-null  float64       
 5   Hour          25277 non-null  float64       
 6   Minute        25277 non-null  float64       
 7   DayOfWeek     25277 non-null  float64       
 8   DayOfYear     25277 non-null  float64       
 9   IsWeekend     25277 non-null  float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 1.9 MB


In [12]:
df.columns

Index(['Thời điểm đo', 'Điện áp (V)', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'DayOfWeek', 'DayOfYear', 'IsWeekend'],
      dtype='object', name=0)

In [13]:
columns_to_convert = df.columns.to_list()
columns_to_convert.remove('Thời điểm đo')
df_preprocess = df[columns_to_convert].astype(float)
df_preprocess.dtypes

0
Điện áp (V)    float64
Year           float64
Month          float64
Day            float64
Hour           float64
Minute         float64
DayOfWeek      float64
DayOfYear      float64
IsWeekend      float64
dtype: object

In [14]:
from sklearn.impute import KNNImputer

# Select the columns to impute
columns_to_impute = ['Year', 'Month', 'Day', 'Hour', 'Minute',
       'DayOfWeek', 'DayOfYear', 'IsWeekend']

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer to the selected columns
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

# Verify the imputation
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25277 entries, 0 to 25276
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Thời điểm đo  25277 non-null  datetime64[ns]
 1   Điện áp (V)   25277 non-null  float64       
 2   Year          25277 non-null  float64       
 3   Month         25277 non-null  float64       
 4   Day           25277 non-null  float64       
 5   Hour          25277 non-null  float64       
 6   Minute        25277 non-null  float64       
 7   DayOfWeek     25277 non-null  float64       
 8   DayOfYear     25277 non-null  float64       
 9   IsWeekend     25277 non-null  float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 1.9 MB


In [15]:
df.columns

Index(['Thời điểm đo', 'Điện áp (V)', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'DayOfWeek', 'DayOfYear', 'IsWeekend'],
      dtype='object', name=0)

In [16]:
columns_to_convert = df.columns.to_list()


In [17]:
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25277 entries, 0 to 25276
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Điện áp (V)  25277 non-null  float64
 1   Year         25277 non-null  float64
 2   Month        25277 non-null  float64
 3   Day          25277 non-null  float64
 4   Hour         25277 non-null  float64
 5   Minute       25277 non-null  float64
 6   DayOfWeek    25277 non-null  float64
 7   DayOfYear    25277 non-null  float64
 8   IsWeekend    25277 non-null  float64
dtypes: float64(9)
memory usage: 1.7 MB


In [18]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)

    return data

df_preprocess = encode(df_preprocess, 'Month', 12)
df_preprocess = encode(df_preprocess, 'Day', 31) 
df_preprocess = encode(df_preprocess, 'Hour', 24)
df_preprocess = encode(df_preprocess, 'Minute', 60)
df_preprocess = encode(df_preprocess, 'DayOfWeek', 7)
df_preprocess = encode(df_preprocess, 'DayOfYear', 366)
# df_preprocess = encode(df_preprocess, 'Quarter', 4)


In [19]:
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25277 entries, 0 to 25276
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Điện áp (V)    25277 non-null  float64
 1   Year           25277 non-null  float64
 2   Month          25277 non-null  float64
 3   Day            25277 non-null  float64
 4   Hour           25277 non-null  float64
 5   Minute         25277 non-null  float64
 6   DayOfWeek      25277 non-null  float64
 7   DayOfYear      25277 non-null  float64
 8   IsWeekend      25277 non-null  float64
 9   Month_sin      25277 non-null  float64
 10  Month_cos      25277 non-null  float64
 11  Day_sin        25277 non-null  float64
 12  Day_cos        25277 non-null  float64
 13  Hour_sin       25277 non-null  float64
 14  Hour_cos       25277 non-null  float64
 15  Minute_sin     25277 non-null  float64
 16  Minute_cos     25277 non-null  float64
 17  DayOfWeek_sin  25277 non-null  float64
 18  DayOfW

In [20]:
df_cols= df_preprocess.columns.to_list()

In [21]:
df_preprocess.dtypes

0
Điện áp (V)      float64
Year             float64
Month            float64
Day              float64
Hour             float64
Minute           float64
DayOfWeek        float64
DayOfYear        float64
IsWeekend        float64
Month_sin        float64
Month_cos        float64
Day_sin          float64
Day_cos          float64
Hour_sin         float64
Hour_cos         float64
Minute_sin       float64
Minute_cos       float64
DayOfWeek_sin    float64
DayOfWeek_cos    float64
DayOfYear_sin    float64
DayOfYear_cos    float64
dtype: object

In [23]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(df_preprocess[['Điện áp (V)', 'Year', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos', 'Minute_sin', 'Minute_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear_sin', 'DayOfYear_cos', 'IsWeekend']])
df_preprocess['Cluster'] = kmeans.labels_
# Balance the sample sizes for each cluster
# min_samples = df_preprocess['Cluster'].value_counts().min()

# # Create a balanced dataframe
# df_preprocess = df_preprocess.groupby('Cluster').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)
# df_preprocess

c:\Users\admin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


AttributeError: 'NoneType' object has no attribute 'split'

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import r2_score

df_preprocess = pd.read_csv('C:/Users/admin/Documents/DPL302m/DPL302m_Material/Code/EduNext_Slot5/df_preprocess.csv')
# Load data
X = df_preprocess.drop(columns=['Điện áp (V)']).values
y = df_preprocess['Điện áp (V)'].values

# Improved data preprocessing
robust_scaler = RobustScaler()  # Better handles outliers
standard_scaler = StandardScaler()
X = robust_scaler.fit_transform(X)
X = standard_scaler.fit_transform(X)
y = standard_scaler.fit_transform(y.reshape(-1, 1)).squeeze()

# Improved graph construction
k = min(5, len(X) - 1)  # Adaptive k based on data size
adj_matrix = kneighbors_graph(
    X, 
    n_neighbors=k, 
    mode='distance',  # Use distance-weighted edges
    include_self=True
)
# Convert distances to similarities
adj_matrix.data = np.exp(-adj_matrix.data)  # Gaussian kernel
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
edge_weight = torch.tensor(adj_matrix.data, dtype=torch.float)

# Convert features and target to PyTorch tensors
x = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

# Create PyTorch Geometric data object
data = Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y)

class GCNMLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, mlp_hidden, out_channels, dropout_rate=0.2):
        super(GCNMLP, self).__init__()
        
        # Increased width of hidden layers
        self.hidden_channels = hidden_channels
        
        # Input projection
        self.input_proj = torch.nn.Linear(in_channels, hidden_channels)
        
        # GCN layers with residual connections
        self.conv1 = GCNConv(hidden_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        
        # Layer Normalization (more stable than Batch Normalization for small batches)
        self.ln1 = torch.nn.LayerNorm(hidden_channels)
        self.ln2 = torch.nn.LayerNorm(hidden_channels)
        self.ln3 = torch.nn.LayerNorm(hidden_channels)
        
        self.global_pool = torch.nn.AdaptiveAvgPool1d(1)

        # MLP with wider layers and skip connections
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, mlp_hidden),
            torch.nn.GELU(),  # GELU activation often works better than ReLU
            torch.nn.Dropout(dropout_rate),
            torch.nn.LayerNorm(mlp_hidden),
            torch.nn.Linear(mlp_hidden, mlp_hidden // 2),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.LayerNorm(mlp_hidden // 2),
            torch.nn.Linear(mlp_hidden // 2, out_channels)
        )
        
        self.dropout_rate = dropout_rate
        
    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
        batch_size = x.size(0)
        # Initial projection
        x = self.input_proj(x)
        identity = x
        
        # First GCN block with residual
        x = self.conv1(x, edge_index, edge_weight)
        x = self.ln1(x)
        x = F.gelu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = x + identity  # Residual connection
        
        # Second GCN block with residual
        identity = x
        x = self.conv2(x, edge_index, edge_weight)
        x = self.ln2(x)
        x = F.gelu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = x + identity  # Residual connection
        
        # Third GCN block with residual
        identity = x
        x = self.conv3(x, edge_index, edge_weight)
        x = self.ln3(x)
        x = F.gelu(x)
        x = x + identity  # Residual connection
        
        x = x.view(batch_size, -1)  
        # MLP layers
        x = self.mlp(x)
        
        return x.squeeze()

# Instantiate model with improved hyperparameters
model = GCNMLP(
    in_channels=X.shape[1],
    hidden_channels=128,  
    mlp_hidden=256,    
    out_channels=1,
    dropout_rate=0.2
)

# Improved optimizer settings
optimizer = torch.optim.AdamW( 
    model.parameters(),
    lr=0.01,           # Lower learning rate
    weight_decay=0.01,  # Increased weight decay
    betas=(0.9, 0.999)  # Default betas
)

# Improved scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=15,
    verbose=True,
    min_lr=1e-6
)

# Custom loss function combining MSE and R² optimization
class R2Loss(torch.nn.Module):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.mse = torch.nn.MSELoss()
    
    def forward(self, pred, true):
        mse_loss = self.mse(pred, true)
        
        # R² component
        ss_tot = torch.sum((true - true.mean()) ** 2)
        ss_res = torch.sum((true - pred) ** 2)
        r2 = 1 - ss_res / (ss_tot + 1e-8)
        
        # Combined loss
        return mse_loss - self.alpha * r2

criterion = R2Loss(alpha=0.1)

# Improved train/test split with stratification
train_mask, test_mask = train_test_split(
    range(len(df_preprocess)),
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Improved training loop
def train(data, train_mask):
    model.train()
    optimizer.zero_grad()
    out = model(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    return loss.item()

# Test function remains the same
def test(data, mask):
    model.eval()
    with torch.no_grad():
        pred = model(data)[mask]
        actual = data.y[mask]
        loss = criterion(pred, actual)
        r2 = r2_score(actual.cpu(), pred.cpu())
    return loss.item(), r2

# Training with improved early stopping
best_loss = float('inf')
best_r2 = float('-inf')
patience = 30
counter = 0
epochs = 500  # Increased epochs

for epoch in range(epochs):
    train_loss = train(data, train_mask)
    test_loss, r2 = test(data, test_mask)
    
    # Learning rate scheduling based on R² score
    scheduler.step(-r2)  # Use negative R² for minimization
    
    # Early stopping based on R² score
    if r2 > best_r2:
        best_r2 = r2
        best_loss = test_loss
        counter = 0
    else:
        counter += 1
    
    if counter >= patience:
        print(f'Early stopping at epoch {epoch}')
        break
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, '
              f'Test Loss: {test_loss:.4f}, R² Score: {r2:.4f}')


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Train Loss: 1.4702, Test Loss: 19.7781, R² Score: -17.5080
Epoch 10, Train Loss: 1.0731, Test Loss: 1.0782, R² Score: -0.0970
Epoch 20, Train Loss: 1.0007, Test Loss: 0.9492, R² Score: 0.0231
Epoch 30, Train Loss: 0.9524, Test Loss: 0.9336, R² Score: 0.0376
Epoch 40, Train Loss: 0.9415, Test Loss: 0.9205, R² Score: 0.0498
Epoch 50, Train Loss: 0.9290, Test Loss: 0.9106, R² Score: 0.0591
Epoch 60, Train Loss: 0.9179, Test Loss: 0.9019, R² Score: 0.0672
Epoch 70, Train Loss: 0.9034, Test Loss: 0.8868, R² Score: 0.0812
Epoch 80, Train Loss: 0.8930, Test Loss: 0.8688, R² Score: 0.0980
Epoch 90, Train Loss: 0.8763, Test Loss: 0.8693, R² Score: 0.0975
Epoch 100, Train Loss: 0.8565, Test Loss: 0.8566, R² Score: 0.1094
Epoch 110, Train Loss: 0.8415, Test Loss: 0.8442, R² Score: 0.1208
Epoch 120, Train Loss: 0.8310, Test Loss: 0.8330, R² Score: 0.1313
Epoch 130, Train Loss: 0.8163, Test Loss: 0.8353, R² Score: 0.1292
Epoch 140, Train Loss: 0.7943, Test Loss: 0.8388, R² Score: 0.1259
Ep